# What is Kubernetes?

Kubernetes coordinates a highly available cluster of computers that are connected to work as a single unit, and automates the distribution and scheduling of application containers across a cluster efficiently.

### Features
 - **Service discovery and load balancing:** TODO
 - **Storage orchestration:** TODO
 - **Automated rollouts and rollbacks:** TODO
 - **Automatic bin packing:** TODO
 - **Self-healing:** TODO
 - **Secret and configuration management:** TODO

### Limitations
 - TODO
 - TODO
 - TODO
 - TODO

# How does Kubernetes work?
 - **Pods:** TODO
 - **Services:** TODO
 - **Ingress controller:** TODO
 - **Volumes:** TODO
 - **Namespaces:** TODO
 - **Secrets and configuration:** TODO

# Let's Get Started!

We will be using a simple web app, created with [Flask](https://flask.palletsprojects.com/en/2.0.x/), to demonstrate the core functionality of Kubernetes. The `app/main.py` script contains code which configures and initializes a local Flask development server. Its contents are included below:

In [1]:
from flask import Flask
app = Flask(__name__)


@app.route("/")
def hello():
    return "Hello from Python/Flask!"


if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.104:5000/ (Press CTRL+C to quit)


Kubernetes allows you to deploy containerized applications to a cluster without tying them specifically to individual machines. A Kubernetes cluster is composed of two types of resources. First, the **Control Plane** handles the management of the cluster, coordinating all activities therein, such as: scheduling applications, maintaining applications' desired state, scaling applications, and rolling out application updates. Second, **Nodes** are physical computers or virtual machines that serve as worker machines in a cluster, running application instances. The following diagram illustrates the architecture of a Kubernetes cluster:

<div style="width: 100%;"><img src="img/module_01_cluster.svg"/></div>

The individual nodes in a cluster communicate with the control plane using the [Kubernetes API](https://kubernetes.io/docs/concepts/overview/kubernetes-api/). If you followed the setup steps for this workshop (located in `README.md`), and did not encounter any issues along the way, you should already have a cluster running locally. Run the following in your terminal/command prompt to view information about active clusters:

In [ ]:
kubectl cluster-info

Then, you can use the following command to view the names, statuses, roles, ages, and versions of individual Nodes:

In [ ]:
kubectl get nodes

Once you have a Kubernetes cluster running, you can begin to deploy your containerized application by creating a **Deployment** configuration. This Deployment tells Kubernetes how to create and update instances of your application. Once a Deployment has been created, the control plane schedules the application instances included in that Deployment to run on individual Nodes in the cluster. A **Deployment Controller** continuously monitors the created application instances, replacing instances that go down or are deleted with an instance running on another Node in the cluster. This provides a self-healing mechanism to address machine failure or maintenance. Now that we have a better idea of what goes on inside individual Nodes in a cluster, let's take a look at an updated cluster diagram.

<div style="width: 100%;"><img src="img/module_02_cluster.svg"/></div>

Before you deploy your Flask application, you need to containerize it. To do so, you must first navigate to the `app/` directory within this workshop folder in a terminal/command prompt window. Starting in the top-level repository directory `.../2022-spring-workshops`, you can use the following command to do so:

In [ ]:
cd "01 - Kubernetes"/app

You then need to point your shell to minikube's docker-daemon by first running the following command:

In [ ]:
minikube docker-env

Next, run the command from the last line of the output in your shell. For example, in the Window's command prompt, run:

In [ ]:
@FOR /f "tokens=*" %i IN ('minikube -p minikube docker-env') DO @%i

Then, you need to create an image of the application (think of this as a snapshot of the app's current state), which can be done with the `docker build` command as follows:

In [ ]:
docker build -f Dockerfile -t flask-app:1.0 .

The `-f` option allows you to specify the filename of the Dockerfile to build the container image from. Using `-t` enables you to name the image and optional tag it with a version in the form `name:tag`. Here, we name the image `flask-app` and tag it as version `1.0`. The `.` at the end of the command specifies the context from which to build the image from, the current working directory in this case. Once your image has been built, you can run the following command to list your Docker images:

In [ ]:
docker image ls

If you see an image with a repository of `flask-app` and a tag of `1.0`, you successfully containerized your Flask application and are ready to deploy it locally with Kubernetes! By default, Kubernetes will attempt to pull Docker images from *the public Docker registry* when creating a Deployment. To avoid this behavior, you need to run a local Docker registry to host your container images. You can start a local Docker registry with the following command:

In [ ]:
docker run -d -p 5000:5000 --restart=always --name registry registry:2

Next, you need to tag the image you created so that you can push it to the local Docker registry you created. This can be achieved with the following command:

In [ ]:
docker tag flask-app:1.0 localhost:5000/flask-app:1.0

Then, push your container image to your local Docker registry with the following command:

In [ ]:
docker push localhost:5000/flask-app:1.0

Finally, you can create a new Deployment from the containerized Flask application by running the following command:

In [ ]:
kubectl create deployment flask-app --image=localhost:5000/flask-app:1.0

Running the `create deployment` command causes the following actions to be performed:
- searching for a suitable node where an instance of the application could be run
- scheduling the application to run on that Node
- configuring the cluster to reschedule the instance on a new Node when needed

You can then list your Deployments with the following command:

In [ ]:
kubectl get deployments

The `READY` column indicates the number of instances of your app that are running, followed by the number of deployments you created. Your output should indicate that there is 1 instance of your app running on the 1 Deployment you created. Currently, your deployed application can only be accessed from inside Kubernetes/kubectl. Later on you will learn how to set up a **Service** to expose your application, but for now you will use a simple proxy server provided by `kubectl`. *Open a new terminal/command prompt window* and run the following command:

In [ ]:
kubectl proxy

You can then view all the APIs hosted through the proxy endpoint with the following command, or by copy-pasting the URL into your web browser:

In [ ]:
curl http://localhost:8001

Run `kubectl get pods` and note the name of the Pod running your `flask-app`. You can then access the pod directly through the API by running the following command, or by copy-pasting the URL into your web browser (make sure to replace `POD-NAME` with the name of the Pod you noted previously): 

In [ ]:
http://localhost:8001/api/v1/namespaces/default/pods/POD-NAME/